In [3]:
import pandas as pd
import os

# Cargar datos
df = pd.read_csv('datos/datos_filtrados_sin_encoding.csv')

# Crear directorio si no existe
if not os.path.exists('latex_tables'):
    os.makedirs('latex_tables')

# Crear tabla de contingencia
combined_data = []

for genus in sorted(df['genus'].unique()):
    genus_data = df[df['genus'] == genus]
    
    # Train por origen
    train_gtdb = len(genus_data[(genus_data['purpose'] == 0) & (genus_data['origen'] == 'gtdb')])
    train_entrez = len(genus_data[(genus_data['purpose'] == 0) & (genus_data['origen'] == 'entrez')])
    
    # Test por origen
    test_gtdb = len(genus_data[(genus_data['purpose'] == 1) & (genus_data['origen'] == 'gtdb')])
    test_entrez = len(genus_data[(genus_data['purpose'] == 1) & (genus_data['origen'] == 'entrez')])
    
    # Descartados
    leftover = len(genus_data[genus_data['purpose'] == 2])
    
    # Total
    total = len(genus_data)
    
    combined_data.append({
        'genus': genus,
        'train_gtdb': train_gtdb,
        'train_entrez': train_entrez,
        'test_gtdb': test_gtdb,
        'test_entrez': test_entrez,
        'leftover': leftover,
        'total': total
    })

# Convertir a DataFrame
table_df = pd.DataFrame(combined_data)

# Calcular totales
totals = {
    'train_gtdb': table_df['train_gtdb'].sum(),
    'train_entrez': table_df['train_entrez'].sum(),
    'test_gtdb': table_df['test_gtdb'].sum(),
    'test_entrez': table_df['test_entrez'].sum(),
    'leftover': table_df['leftover'].sum(),
    'total': table_df['total'].sum()
}

# Generar código LaTeX
latex_lines = [
    "\\begin{table}[htbp]",
    "\\centering",
    "\\caption{Distribución de secuencias por género bacteriano, origen y propósito}",
    "\\label{tab:dataset_distribution}",
    "\\setlength{\\tabcolsep}{4pt}",
    "\\renewcommand{\\arraystretch}{1.2}",
    "\\begin{tabular}{l|cc|cc|c|c}",
    "\\hline",
    "& \\multicolumn{2}{c|}{\\textbf{Train}} & \\multicolumn{2}{c|}{\\textbf{Test}} & & \\\\",
    "\\textbf{Género} & \\textbf{GTDB} & \\textbf{Entrez} & \\textbf{GTDB} & \\textbf{Entrez} & \\textbf{Leftover} & \\textbf{Total} \\\\",
    "\\hline"
]

# Agregar filas de datos
for _, row in table_df.iterrows():
    genus = row['genus'].replace('_', '\\_')
    train_gtdb = row['train_gtdb'] if row['train_gtdb'] > 0 else '--'
    train_entrez = row['train_entrez'] if row['train_entrez'] > 0 else '--'
    test_gtdb = row['test_gtdb'] if row['test_gtdb'] > 0 else '--'
    test_entrez = row['test_entrez'] if row['test_entrez'] > 0 else '--'
    leftover = row['leftover'] if row['leftover'] > 0 else '--'
    total = row['total']
    
    latex_lines.append(f"{genus} & {train_gtdb} & {train_entrez} & {test_gtdb} & {test_entrez} & {leftover} & {total} \\\\")

# Calcular estadísticas para la tabla
total_genera = len(table_df)
total_train = totals['train_gtdb'] + totals['train_entrez']
total_test = totals['test_gtdb'] + totals['test_entrez']
total_used = total_train + total_test

# Agregar totales y estadísticas dentro de la tabla
latex_lines.extend([
    "\\hline",
    f"\\textbf{{Total}} & \\textbf{{{totals['train_gtdb']}}} & \\textbf{{{totals['train_entrez']}}} & \\textbf{{{totals['test_gtdb']}}} & \\textbf{{{totals['test_entrez']}}} & \\textbf{{{totals['leftover']}}} & \\textbf{{{totals['total']}}} \\\\",
    "\\hline",
    f"\\multicolumn{{7}}{{l}}{{\\textbf{{Distribución: {total_genera} géneros bacterianos procesados}}}} \\\\",
    f"\\multicolumn{{7}}{{l}}{{\\textbf{{Total entrenamiento: {total_train:,} secuencias}}}} \\\\",
    f"\\multicolumn{{7}}{{l}}{{\\textbf{{Total evaluación: {total_test:,} secuencias}}}} \\\\",
    f"\\multicolumn{{7}}{{l}}{{\\textbf{{Total utilizadas: {total_used:,} secuencias}}}} \\\\",
    "\\hline",
    "\\end{tabular}",
    "\\end{table}"
])

# Unir código LaTeX
latex_content = '\n'.join(latex_lines)

# Guardar archivo
filepath = os.path.join('latex_tables', 'dataset_distribution.tex')
with open(filepath, 'w', encoding='utf-8') as f:
    f.write(latex_content)

# Imprimir solo el código LaTeX
print(latex_content)

\begin{table}[htbp]
\centering
\caption{Distribución de secuencias por género bacteriano, origen y propósito}
\label{tab:dataset_distribution}
\setlength{\tabcolsep}{4pt}
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{l|cc|cc|c|c}
\hline
& \multicolumn{2}{c|}{\textbf{Train}} & \multicolumn{2}{c|}{\textbf{Test}} & & \\
\textbf{Género} & \textbf{GTDB} & \textbf{Entrez} & \textbf{GTDB} & \textbf{Entrez} & \textbf{Leftover} & \textbf{Total} \\
\hline
Acinetobacter & 160 & 40 & 9 & 291 & 564 & 1064 \\
Arthrobacter & 167 & 33 & 6 & 294 & 625 & 1125 \\
Bifidobacterium & 120 & 80 & 4 & 296 & 543 & 1043 \\
Bradyrhizobium & 181 & 19 & 48 & 252 & 683 & 1183 \\
Chryseobacterium & 128 & 72 & 41 & 259 & 558 & 1058 \\
Collinsella & 139 & 61 & 9 & 207 & -- & 416 \\
Corynebacterium & 132 & 68 & 58 & 242 & 558 & 1058 \\
Flavobacterium & 185 & 15 & 147 & 153 & 744 & 1244 \\
Mesorhizobium & 141 & 59 & -- & 300 & 587 & 1087 \\
Methylobacterium & 93 & 107 & 4 & 296 & 501 & 1001 \\
Microbacterium & 177 & 

In [2]:
import pandas as pd
import os

# Cargar datos
df = pd.read_csv('datos/datos_filtrados_sin_encoding.csv')

# Crear directorio si no existe
if not os.path.exists('latex_tables'):
    os.makedirs('latex_tables')

# Crear tabla de contingencia
combined_data = []

for genus in sorted(df['genus'].unique()):
    genus_data = df[df['genus'] == genus]
    
    # Train por origen
    train_gtdb = len(genus_data[(genus_data['purpose'] == 0) & (genus_data['origen'] == 'gtdb')])
    train_entrez = len(genus_data[(genus_data['purpose'] == 0) & (genus_data['origen'] == 'entrez')])
    
    # Test por origen
    test_gtdb = len(genus_data[(genus_data['purpose'] == 1) & (genus_data['origen'] == 'gtdb')])
    test_entrez = len(genus_data[(genus_data['purpose'] == 1) & (genus_data['origen'] == 'entrez')])
    
    # Descartados
    leftover = len(genus_data[genus_data['purpose'] == 2])
    
    # Total
    total = len(genus_data)
    
    combined_data.append({
        'genus': genus,
        'train_gtdb': train_gtdb,
        'train_entrez': train_entrez,
        'test_gtdb': test_gtdb,
        'test_entrez': test_entrez,
        'leftover': leftover,
        'total': total
    })

# Convertir a DataFrame
table_df = pd.DataFrame(combined_data)

# Obtener lista de géneros ordenados
genera = sorted(df['genus'].unique())
genera_escaped = [genus.replace('_', '\\_') for genus in genera]

# Calcular totales por categoría
totals = {
    'train_gtdb': table_df['train_gtdb'].sum(),
    'train_entrez': table_df['train_entrez'].sum(),
    'test_gtdb': table_df['test_gtdb'].sum(),
    'test_entrez': table_df['test_entrez'].sum(),
    'leftover': table_df['leftover'].sum(),
    'total': table_df['total'].sum()
}

# Generar código LaTeX transpuesto
num_genera = len(genera)
column_spec = 'l|' + 'c' * num_genera + '|c'

latex_lines = [
    "\\begin{table}[htbp]",
    "\\centering",
    "\\caption{Distribución de secuencias por género bacteriano, origen y propósito (transpuesta)}",
    "\\label{tab:dataset_distribution_transposed}",
    "\\setlength{\\tabcolsep}{3pt}",
    "\\renewcommand{\\arraystretch}{1.2}",
    "\\resizebox{\\textwidth}{!}{%",
    f"\\begin{{tabular}}{{{column_spec}}}",
    "\\hline",
    "\\rule{0pt}{3ex}" # Espacio extra para acomodar texto rotado
]

# Encabezados de columnas (géneros rotados)
header_line = "\\textbf{Categoría} & " + " & ".join([f"\\rotatebox{{90}}{{\\textbf{{{genus}}}}}" for genus in genera_escaped]) + " & \\textbf{Total} \\\\"
latex_lines.extend([
    header_line,
    "\\hline",
    "\\rule{0pt}{2.5ex}" # Espacio después del header
])

# Crear diccionario para acceso rápido por género
genus_data_dict = {row['genus']: row for _, row in table_df.iterrows()}

# Categorías a mostrar
categories = [
    ('Train GTDB', 'train_gtdb'),
    ('Train Entrez', 'train_entrez'),
    ('Test GTDB', 'test_gtdb'),
    ('Test Entrez', 'test_entrez'),
    ('Leftover', 'leftover'),
    ('Total', 'total')
]

# Agregar filas de datos
for cat_name, cat_key in categories:
    values = []
    for genus in genera:
        value = genus_data_dict[genus][cat_key]
        values.append(str(value) if value > 0 else '--')
    
    total_value = totals[cat_key]
    row_line = f"\\textbf{{{cat_name}}} & " + " & ".join(values) + f" & \\textbf{{{total_value}}} \\\\"
    latex_lines.append(row_line)

# Calcular estadísticas para la tabla
total_genera = len(table_df)
total_train = totals['train_gtdb'] + totals['train_entrez']
total_test = totals['test_gtdb'] + totals['test_entrez']
total_used = total_train + total_test

# Agregar estadísticas al final
latex_lines.extend([
    "\\hline",
    f"\\multicolumn{{{num_genera + 2}}}{{l}}{{\\textbf{{Géneros procesados: {total_genera}}}}} \\\\",
    f"\\multicolumn{{{num_genera + 2}}}{{l}}{{\\textbf{{Total entrenamiento: {total_train:,} secuencias}}}} \\\\",
    f"\\multicolumn{{{num_genera + 2}}}{{l}}{{\\textbf{{Total evaluación: {total_test:,} secuencias}}}} \\\\",
    f"\\multicolumn{{{num_genera + 2}}}{{l}}{{\\textbf{{Total utilizadas: {total_used:,} secuencias}}}} \\\\",
    "\\hline",
    "\\end{tabular}%",
    "}",
    "\\end{table}"
])

# Unir código LaTeX
latex_content = '\n'.join(latex_lines)

# Guardar archivo
filepath = os.path.join('latex_tables', 'dataset_distribution_transposed.tex')
with open(filepath, 'w', encoding='utf-8') as f:
    f.write(latex_content)

# Imprimir solo el código LaTeX
print(latex_content)

\begin{table}[htbp]
\centering
\caption{Distribución de secuencias por género bacteriano, origen y propósito (transpuesta)}
\label{tab:dataset_distribution_transposed}
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.2}
\resizebox{\textwidth}{!}{%
\begin{tabular}{l|ccccccccccccccccccccccccccccc|c}
\hline
\rule{0pt}{3ex}
\textbf{Categoría} & \rotatebox{90}{\textbf{Acinetobacter}} & \rotatebox{90}{\textbf{Arthrobacter}} & \rotatebox{90}{\textbf{Bifidobacterium}} & \rotatebox{90}{\textbf{Bradyrhizobium}} & \rotatebox{90}{\textbf{Chryseobacterium}} & \rotatebox{90}{\textbf{Collinsella}} & \rotatebox{90}{\textbf{Corynebacterium}} & \rotatebox{90}{\textbf{Flavobacterium}} & \rotatebox{90}{\textbf{Mesorhizobium}} & \rotatebox{90}{\textbf{Methylobacterium}} & \rotatebox{90}{\textbf{Microbacterium}} & \rotatebox{90}{\textbf{Micromonospora}} & \rotatebox{90}{\textbf{Mycobacterium}} & \rotatebox{90}{\textbf{Nocardia}} & \rotatebox{90}{\textbf{Nocardioides}} & \rotatebox{90}{\textbf{Novo

In [16]:
import duckdb
import pandas as pd
import os

def generar_tabla_latex_encodings(archivo_csv):
    """
    Genera una tabla LaTeX con estadísticas de tiempo y memoria para cada encoding.
    """
    
    # Leer los datos
    resultados = pd.read_csv(archivo_csv)
    
    # Ejecutar consulta SQL para obtener estadísticas
    stats_df = duckdb.sql("""
    SELECT 
        encoding,
        COUNT(*) as num_ejecuciones,
        
        -- Estadísticas de tiempo
        ROUND(AVG(time), 3) as tiempo_promedio_seg,
        ROUND(STDDEV(time), 3) as tiempo_desv_std,
        ROUND(MIN(time), 3) as tiempo_min,
        ROUND(MAX(time), 3) as tiempo_max,
        
        -- Estadísticas de memoria
        ROUND(AVG(memory_used), 2) as memoria_promedio_mb,
        ROUND(STDDEV(memory_used), 2) as memoria_desv_std,
        ROUND(MIN(memory_used), 2) as memoria_min_mb,
        ROUND(MAX(memory_used), 2) as memoria_max_mb,
        
        -- Eficiencia (memoria/tiempo)
        ROUND(AVG(memory_used/time), 2) as eficiencia_mb_por_seg
        
    FROM resultados
    GROUP BY encoding
    ORDER BY tiempo_promedio_seg ASC;
    """).to_df()
    
    # Obtener número total de iteraciones para la nota
    total_iteraciones = resultados['iter'].nunique()
    
    # Generar tabla LaTeX
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Estadísticas de Tiempo y Memoria por Encoding}
\\label{tab:encoding_stats}
\\scriptsize
\\begin{tabular}{lccc}
\\toprule
& \\multicolumn{2}{c}{\\textbf{Tiempo (segundos)}} & \\textbf{Memoria} \\\\
\\cmidrule(lr){2-3} \\cmidrule(lr){4-4}
\\textbf{Encoding} & Promedio & Desv. Std & (MB) \\\\
\\midrule
"""
    
    # Agregar filas de datos
    for _, row in stats_df.iterrows():
        encoding = row['encoding'].replace('_', '\\_')  # Escapar guiones bajos para LaTeX
        
        latex_code += f"{encoding} & "
        latex_code += f"{row['tiempo_promedio_seg']:.2f} & "
        latex_code += f"{row['tiempo_desv_std']:.2f} & "
        latex_code += f"{row['memoria_promedio_mb']:.2f} \\\\\n"
    
    # Cerrar tabla
    latex_code += """\\bottomrule
\\end{tabular}
"""
    
    # Agregar nota sobre iteraciones
    latex_code += f"""\\\\[0.5em]
\\footnotesize
\\textit{{Nota: Cada encoding fue ejecutado {total_iteraciones} veces para el análisis de tiempo. El uso de memoria es constante para cada encoding y no presenta varianza entre ejecuciones.}}
\\end{{table*}}"""
    
    return latex_code, stats_df

# Ejecutar función principal
archivo_csv = "datos/resultados_codificacion_tiempo_memoria.csv"
tabla_latex, df_estadisticas = generar_tabla_latex_encodings(archivo_csv)

# Mostrar tabla LaTeX
print("TABLA LATEX GENERADA:")
print("=" * 60)
print(tabla_latex)
print("=" * 60)

# Mostrar también los datos en formato DataFrame para verificación
print("\nDATOS ESTADÍSTICOS:")
print(df_estadisticas.to_string(index=False))

# Crear directorio si no existe
import os
os.makedirs("latex_tables", exist_ok=True)

# Guardar en archivo
with open("latex_tables/tabla_encodings_tiempo_memoria.tex", "w", encoding="utf-8") as f:
    f.write(tabla_latex)

print(f"\nTabla guardada en: latex_tables/tabla_encodings_tiempo_memoria.tex")

TABLA LATEX GENERADA:
\begin{table*}[htbp]
\centering
\caption{Estadísticas de Tiempo y Memoria por Encoding}
\label{tab:encoding_stats}
\scriptsize
\begin{tabular}{lccc}
\toprule
& \multicolumn{2}{c}{\textbf{Tiempo (segundos)}} & \textbf{Memoria} \\
\cmidrule(lr){2-3} \cmidrule(lr){4-4}
\textbf{Encoding} & Promedio & Desv. Std & (MB) \\
\midrule
PS\_FFT & 2.01 & 0.01 & 188.69 \\
PS\_Wavelet & 2.48 & 0.01 & 378.29 \\
PS\_K-mers & 2.95 & 0.07 & 423.57 \\
PS\_One Hot & 4.33 & 0.01 & 1886.86 \\
PS\_K-mers + FFT & 4.34 & 0.04 & 188.45 \\
PS\_K-mers + Wavelet & 4.68 & 0.05 & 378.06 \\
PS\_One Hot + Wavelet & 5.64 & 0.01 & 1887.33 \\
PS\_One Hot + FFT & 5.86 & 0.01 & 943.33 \\
AS\_Wavelet & 9.78 & 0.06 & 1849.13 \\
AS\_FFT & 14.15 & 0.08 & 923.99 \\
AS\_K-mers & 14.17 & 0.34 & 1981.90 \\
AS\_One Hot & 20.17 & 0.41 & 9239.87 \\
AS\_K-mers + Wavelet & 20.35 & 0.21 & 1848.89 \\
AS\_One Hot + Wavelet & 24.44 & 0.12 & 9241.99 \\
AS\_K-mers + FFT & 25.25 & 0.24 & 923.75 \\
AS\_One Hot + FFT & 57.4

In [2]:
import duckdb
import pandas as pd
import os

# Leer los datos
resultados = pd.read_csv("results/training_times/training_times_complete_detailed.csv")

# Ejecutar consulta SQL para obtener estadísticas
stats_df = duckdb.sql("""
SELECT 
    modelo,
    encoding,
    COUNT(*) as num_ejecuciones,
    ROUND(AVG(tiempo), 2) as tiempo_promedio_seg,
    ROUND(STDDEV(tiempo), 2) as tiempo_desv_std,
    ROUND(MIN(tiempo), 2) as tiempo_min,
    ROUND(MAX(tiempo), 2) as tiempo_max
FROM resultados
GROUP BY modelo, encoding
-- ORDER BY modelo ASC, tiempo_promedio_seg ASC;
ORDER BY tiempo_promedio_seg ASC;
""").to_df()

# Obtener número total de iteraciones para la nota
total_iteraciones = resultados['iter'].nunique()

# Generar tabla LaTeX
latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Tiempos de Entrenamiento por Modelo y Encoding}
\\label{tab:training_times}
\\scriptsize
\\begin{tabular}{llcc}
\\toprule
& & \\multicolumn{2}{c}{\\textbf{Tiempo (segundos)}} \\\\
\\cmidrule(lr){3-4}
\\textbf{Modelo} & \\textbf{Encoding} & Promedio & Desv. Std \\\\
\\midrule
"""

# Agregar filas de datos
for _, row in stats_df.iterrows():
    modelo = row['modelo']
    encoding = row['encoding'].replace('_', '\\_')  # Escapar guiones bajos para LaTeX
    
    latex_code += f"{modelo} & {encoding} & "
    latex_code += f"{row['tiempo_promedio_seg']:.2f} & "
    latex_code += f"{row['tiempo_desv_std']:.2f} \\\\\n"

# Cerrar tabla
latex_code += """\\bottomrule
\\end{tabular}
"""

# Agregar nota sobre iteraciones
latex_code += f"""\\\\[0.5em]
\\footnotesize
\\textit{{Nota: Cada combinación modelo-encoding fue ejecutada {total_iteraciones} veces para el análisis de tiempos de entrenamiento.}}
\\end{{table*}}"""

# Mostrar tabla LaTeX
print("TABLA LATEX GENERADA:")
print("=" * 60)
print(latex_code)
print("=" * 60)

# Mostrar también los datos en formato DataFrame para verificación
print("\nDATOS ESTADÍSTICOS:")
print(stats_df.to_string(index=False))

# Crear directorio si no existe
os.makedirs("latex_tables", exist_ok=True)

# Guardar en archivo
with open("latex_tables/tabla_tiempos_entrenamiento.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)

print(f"\nTabla guardada en: latex_tables/tabla_tiempos_entrenamiento.tex")

TABLA LATEX GENERADA:
\begin{table*}[htbp]
\centering
\caption{Tiempos de Entrenamiento por Modelo y Encoding}
\label{tab:training_times}
\scriptsize
\begin{tabular}{llcc}
\toprule
& & \multicolumn{2}{c}{\textbf{Tiempo (segundos)}} \\
\cmidrule(lr){3-4}
\textbf{Modelo} & \textbf{Encoding} & Promedio & Desv. Std \\
\midrule
RandomForest & AS\_Wavelet & 1.74 & 0.00 \\
RandomForest & AS\_One Hot + Wavelet & 2.64 & 0.00 \\
SVM & PS\_K-mers + FFT & 2.94 & 0.01 \\
SVM & PS\_FFT & 3.22 & 0.01 \\
RandomForest & PS\_Wavelet & 4.50 & 0.02 \\
RandomForest & AS\_K-mers + Wavelet & 4.98 & 0.10 \\
RandomForest & PS\_K-mers & 5.28 & 0.11 \\
RandomForest & AS\_K-mers & 5.57 & 0.00 \\
RandomForest & PS\_One Hot + Wavelet & 5.66 & 0.04 \\
RandomForest & PS\_K-mers + Wavelet & 6.26 & 0.09 \\
RandomForest & PS\_One Hot & 6.28 & 0.03 \\
RandomForest & AS\_One Hot & 7.68 & 0.10 \\
SVM & PS\_Wavelet & 12.07 & 0.02 \\
SVM & PS\_K-mers + Wavelet & 13.60 & 0.41 \\
RandomForest & PS\_K-mers + FFT & 13.96 & 0.25 